# Modelo de Machine Learning

In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score
import pickle

In [3]:
df = pd.read_csv('crop_recommendation.csv')

## Análisis y preprocesamiento de los datos

In [4]:
# Primeras 5 filas del conjunto de datos
df.head()

,N,P,K,temperature,humidity,ph,rainfall,label
0,90,42,43,20.879744,82.002744,6.502985,202.935536,rice
1,85,58,41,21.770462,80.319644,7.038096,226.655537,rice
2,60,55,44,23.004459,82.320763,7.840207,263.964248,rice
3,74,35,40,26.491096,80.158363,6.980401,242.864034,rice
4,78,42,42,20.130175,81.604873,7.628473,262.717340,rice


In [5]:
# Tamaño del dataset
df.shape

(2200, 8)

Tenemos un dataset formado por 2200 instancias con 8 características cada una.

In [6]:
# Tipo de dato en cada columna
df.dtypes

N                int64
P                int64
K                int64
temperature    float64
humidity       float64
ph             float64
rainfall       float64
label           object
dtype: object

In [7]:
# Número de clases diferentes
df['label'].nunique()

22

In [8]:
# Número de instancias de cada clase
df['label'].value_counts()

label
rice           100
maize          100
jute           100
cotton         100
coconut        100
papaya         100
orange         100
apple          100
muskmelon      100
watermelon     100
grapes         100
mango          100
banana         100
pomegranate    100
lentil         100
blackgram      100
mungbean       100
mothbeans      100
pigeonpeas     100
kidneybeans    100
chickpea       100
coffee         100
Name: count, dtype: int64

El dataset está perfectamente equilibrado, habiendo 100 instancias de cada una de las 22 clases.

In [9]:
# Principales estadísticoss de los datos numéricos
df.describe()

,N,P,K,temperature,humidity,ph,rainfall
count,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000,2200.000000
mean,50.551818,53.362727,48.149091,25.616244,71.481779,6.469480,103.463655
std,36.917334,32.985883,50.647931,5.063749,22.263812,0.773938,54.958389
min,0.000000,5.000000,5.000000,8.825675,14.258040,3.504752,20.211267
25%,21.000000,28.000000,20.000000,22.769375,60.261953,5.971693,64.551686
50%,37.000000,51.000000,32.000000,25.598693,80.473146,6.425045,94.867624
75%,84.250000,68.000000,49.000000,28.561654,89.948771,6.923643,124.267508
max,140.000000,145.000000,205.000000,43.675493,99.981876,9.935091,298.560117


In [10]:
# Valores promedio de las características de cada tipo de cultivo
df.groupby('label').mean()

,N,P,K,temperature,humidity,ph,rainfall
label,,,,,,,
apple,20.80,134.22,199.89,22.630942,92.333383,5.929663,112.654779
banana,100.23,82.01,50.05,27.376798,80.358123,5.983893,104.626980
blackgram,40.02,67.47,19.24,29.973340,65.118426,7.133952,67.884151
chickpea,40.09,67.79,79.92,18.872847,16.860439,7.336957,80.058977
coconut,21.98,16.93,30.59,27.409892,94.844272,5.976562,175.686646
coffee,101.20,28.74,29.94,25.540477,58.869846,6.790308,158.066295
cotton,117.77,46.24,19.56,23.988958,79.843474,6.912675,80.398043
grapes,23.18,132.53,200.11,23.849575,81.875228,6.025937,69.611829
jute,78.40,46.86,39.99,24.958376,79.639864,6.732778,174.792798


In [11]:
# Valores faltantes
df.isna().sum()

N              0
P              0
K              0
temperature    0
humidity       0
ph             0
rainfall       0
label          0
dtype: int64

No hay valores faltantes.

In [12]:
# Separamos la variable dependiente de las independientes
X = df[df.columns[:-1]]
Y = df['label']
# Separamos en conjunto de entrenamiento y de validación.
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=7)

In [13]:
# Reescalamos los datos
scaler = MinMaxScaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

## Creación del modelo

Vamos a crear un modelo de machine learning mediante al algoritmo k-nearest neighbors. Este algoritmo tiene varios hiperparámetros, entre ellos el número de vecinos, la métrica de distancia y la función de peso. Para determinar qué hiperparametros maximizan la precisión de nuestro modelo, haremos uso de una búsqueda en rejilla.

In [14]:
param_grid = {'n_neighbors' : list(range(1,31)),
              'weights' : ['uniform','distance'],
              'metric' : ['minkowski','euclidean','manhattan']}

grid_search = GridSearchCV(KNeighborsClassifier(), param_grid, cv=5, n_jobs=-1, scoring='accuracy')
grid_search.fit(X_train,Y_train)

GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'metric': ['minkowski', 'euclidean', 'manhattan'],
                         'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30],
                         'weights': ['uniform', 'distance']},
             scoring='accuracy')

In [15]:
print(f'Mejores parámetros para nuestro modelo: {grid_search.best_params_}')

Mejores parámetros para nuestro modelo: {'metric': 'manhattan', 'n_neighbors': 5, 'weights': 'distance'}


Una vez conocidos los hiperparámetros que maximizan el rendimiento del modelo, procedemos a crear nuestro modelo usando dichos hiperparámetros.

In [16]:
knn = KNeighborsClassifier(n_neighbors=5,weights='distance',metric='manhattan')

In [17]:
# Entrenamos el modelo
knn.fit(X_train,Y_train)

KNeighborsClassifier(metric='manhattan', weights='distance')

In [18]:
# Evaluamos el modelo con los datos de validación
Y_pred = knn.predict(X_test)
print(f'Exactitud (accuracy) del modelo: {round(accuracy_score(Y_test,Y_pred),3)}')

Exactitud (accuracy) del modelo: 0.989


Como vemos, la exactitud del modelo es casi perfecta, lo cual garantizará casi con total seguiridad que la predicciones serán válidas para el usuario de la aplicación.

In [19]:
# Usamos el módulo pickle para serializar el modelo de machine learning, que usaremos en el script
# de la aplicación
with open('pickle_objects/knn.pickle', 'wb') as f:
    pickle.dump(knn, f)

In [20]:
# Hacemos lo mismo con el objeto scaler
with open('pickle_objects/scaler.pickle','wb') as f:
    pickle.dump(scaler, f)

In [40]:
# Y también con el DataFrame
with open('pickle_objects/df.pickle','wb') as f:
    pickle.dump(df, f)